<a href="https://colab.research.google.com/github/Seungmin-leekorean/chartmania/blob/master/CNN_prac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications


In [ ]:
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = './imagess'
validation_data_dir = './tests'
nb_train_samples = 48
nb_validation_samples = 4
epochs = 100
batch_size = 4


In [ ]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save('bottleneck_features_train.npy',
            bottleneck_features_train)
    print(generator)
    

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save('bottleneck_features_validation.npy',
            bottleneck_features_validation)

In [ ]:

def train_top_model():
    train_data = np.load('bottleneck_features_train.npy')
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load('bottleneck_features_validation.npy')
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    a = model.output
    print(a)
    print(validation_labels)

In [ ]:
save_bottlebeck_features()


Found 48 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [ ]:
train_top_model()

Train on 48 samples, validate on 4 samples
Epoch 1/100
48/48 [==============================] - 0s 4ms/step - loss: 3.1549 - accuracy: 0.5833 - val_loss: 2.8140 - val_accuracy: 0.5000
Epoch 2/100
48/48 [==============================] - 0s 1ms/step - loss: 1.9875 - accuracy: 0.5208 - val_loss: 1.0581 - val_accuracy: 0.2500
Epoch 3/100
48/48 [==============================] - 0s 2ms/step - loss: 1.2910 - accuracy: 0.6042 - val_loss: 1.2580 - val_accuracy: 0.0000e+00
Epoch 4/100
48/48 [==============================] - 0s 1ms/step - loss: 0.8051 - accuracy: 0.6875 - val_loss: 1.5155 - val_accuracy: 0.5000
Epoch 5/100
48/48 [==============================] - 0s 1ms/step - loss: 1.1023 - accuracy: 0.6875 - val_loss: 1.6600 - val_accuracy: 0.5000
Epoch 6/100
48/48 [==============================] - 0s 1ms/step - loss: 0.3944 - accuracy: 0.8333 - val_loss: 2.3977 - val_accuracy: 0.5000
Epoch 7/100
48/48 [==============================] - 0s 1ms/step - loss: 0.8039 - accuracy: 0.7708 - val_lo

In [ ]:
print(validation_data_dir)

./tests
